In [ ]:
##USING GROQ
import os
from dotenv import load_dotenv
load_dotenv()

groq_api_key=os.getenv("GROQ_API_KEY")
groq_api_key

In [ ]:
from langchain_groq import ChatGroq
model=ChatGroq(model="llama-3.1-8b-instant",groq_api_key=groq_api_key)
model

In [ ]:
from langchain_core.messages import HumanMessage

model.invoke([HumanMessage(content="Hello, my name is varun and I am an ai engineer")])

In [ ]:
from langchain_core.messages import AIMessage
model.invoke([
    HumanMessage(content="Hello, my name is varun and I am an ai engineer"),
    AIMessage(content="Hello Varun, nice to meet you. As an AI engineer, I'm sure you're working on some exciting projects. What area of AI engineering are you currently focused on? Are you working on natural language processing, computer vision, or perhaps robotics?"),
    HumanMessage(content="hey whats my name and what do I do?")
])

# Mesage history
We can use a Message History class to wrap our model and make it stateful. This will keep track of inputs and outputs of the model, and store them in some datastore. Future interactions will then load those messages and pass them into the chain as part of the input. Let's see how to use this!

In [ ]:
from langchain_community.chat_message_histories import ChatMessageHistory
from langchain_core.chat_history import  BaseChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory


store={}
def get_session_history(session_id:str)->BaseChatMessageHistory:
    if session_id not in store:
        store[session_id]=ChatMessageHistory()
    return store[session_id]

with_message_history=RunnableWithMessageHistory(model,get_session_history)

In [ ]:
config={"configurable":{"session_id":"chat1"}}

In [ ]:
response=with_message_history.invoke(
    [HumanMessage(content="Hi, My name is varu and i am an ai enginner")],
    config=config
)

In [ ]:
response.content

In [ ]:
## change the config --> session id
config1={"configurable":{"session_id":"chat2"}}
response=with_message_history.invoke(
    [HumanMessage(content="whats my name?")],
    config=config1
)
response.content

In [ ]:
response=with_message_history.invoke(
    [HumanMessage(content="Hey mu name is john")],
    config=config1
)
response.content

In [ ]:
response=with_message_history.invoke(
    [HumanMessage(content="whats my name?")],
    config=config1
)
response.content

# WORKING WITH Prompt Template

In [ ]:
from langchain_core.prompts import ChatPromptTemplate,MessagesPlaceholder
prompt=ChatPromptTemplate.from_messages(
    [
        ("system","You are a helpfull assistant.Answer all the questions to the most of your ability"),
        MessagesPlaceholder(variable_name="messages")
    ]
)
chain=prompt|model

In [ ]:
chain.invoke({"messages":[HumanMessage(content="Hi my name is Varun")]})

In [ ]:
with_message_history=RunnableWithMessageHistory(chain,get_session_history)

In [ ]:
config={"configurable":{"session_id":"chat13"}}
response=with_message_history.invoke(
    [HumanMessage(content="Hi my name is Varun")],
     config=config )
response.content

In [ ]:
prompt=ChatPromptTemplate.from_messages(
    [
        ("system","You are a helpfull assistant.Answer all the questions to the most of your ability in {language}"),
        MessagesPlaceholder(variable_name="messages")
    ]
)
chain=prompt|model

In [ ]:
response=chain.invoke({"messages":[HumanMessage(content="Hi my name is Varun")],"language":"Hindi"})
response.content

In [ ]:
with_message_history=RunnableWithMessageHistory(
    chain,
    get_session_history,
    input_messages_key="messages"
)

In [ ]:
config={"configurable":{"session_id":"chat4"}}
response=with_message_history.invoke(
    {"messages":[HumanMessage(content="Hi ia am varun")],"language":"Hindi"},
    config=config
)
response.content

# Managing the Conversation History

One important concept to understand when building chatbots is how to manage conversation history. If left unmanaged, the list of messages will grow unbounded and potentially overflow the context window of the LLM. Therefore, it is important to add a step that limits the size of the messages you are passing in.


In [ ]:
from langchain_core.messages import SystemMessage,trim_messages
trimmer=trim_messages(
    max_tokens=45,
    strategy="last",
    token_counter=model,
    include_system=True,
    allow_partial=False,
    start_on="human"
)
messages = [
    SystemMessage(content="you're a good assistant"),
    HumanMessage(content="hi I'm bob"),

    AIMessage(content="hi!"),
    HumanMessage(content="I like vanilla ice cream"),
    AIMessage(content="nice"),

    HumanMessage(content="whats 2 + 2"),
    AIMessage(content="4"),
    HumanMessage(content="thanks"),

    AIMessage(content="no problem!"),
    HumanMessage(content="having fun?"),
    AIMessage(content="yes!")
]

trimmer.invoke(messages)


In [ ]:
from operator import itemgetter

from langchain_core.runnables import RunnablePassthrough

chain=(
    RunnablePassthrough.assign(messages=itemgetter("messages")|trimmer)
    | prompt
    | model
)
response=chain.invoke({
    "messages":messages + [HumanMessage(content="what ice cream do i like?")],
    "language":"English"
})
response.content

In [ ]:
response=chain.invoke({
    "messages":messages + [HumanMessage(content="what math problem did i asked for?")],
    "language":"English"
})
response.content

In [ ]:
with_message_history=RunnableWithMessageHistory(
    chain,
    get_session_history,
    input_messages_key="messages"
)
config={"configurable":{"session_id":"chat5"}}